In [1]:
import langchain
print(langchain.__version__)

1.1.1


In [2]:
!nvidia-smi

Wed Dec 10 13:50:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.07                 Driver Version: 566.07         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   56C    P8              6W /   85W |     643MiB /   8188MiB |     16%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import os, sys
from dotenv import load_dotenv
from typing import Annotated, Literal, TypedDict

from telethon import TelegramClient, errors
import asyncio
import re
import nltk
from nltk.corpus import stopwords
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages
from langchain_core.messages import SystemMessage

from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


import torch
print(f"PyTorch версия: {torch.__version__}")
print(f"Версия CUDA в PyTorch: {torch.version.cuda}")
print("Доступность cuda", torch.cuda.is_available())

nltk.download("stopwords")
sw = stopwords.words("russian")

load_dotenv()

api_id = os.getenv('APP_ID_TG')
api_hash = os.getenv('API_HASH_TG')
mistral_api_key = os.getenv('MISTRAL_API_KEY')
channels = [
    "@tass_agency",
    "@rian_ru",
    "@kommersant",
    "@gazeta_ru",
    "@meduzalive",
    "@rbc_news"
]

PyTorch версия: 2.9.1+cu126
Версия CUDA в PyTorch: 12.6
Доступность cuda True


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 1 Подготовка данных

In [4]:
def clean_text_ru(text): 
   # Приведение текста к нижнему регистру
    text = text.lower()
    # Замена всех не-словесных символов на пробел (кроме букв и знаков препинания)
    text = re.sub(r'\W+', ' ', text)

    # Удаление URL-адресов
    text = re.sub(r"http\S+", "", text)

    # Создание шаблона для HTML-тегов
    html = re.compile(r'&lt;.*?&gt;')

    # Удаление HTML-тегов из текста
    text = html.sub(r'', text)

    # Список пунктуаций для удаления
    punctuations = '@#!?+&amp;*[]-%.:/();$=&gt;&lt;|{}^' + "'`" + '_'
    for p in punctuations:
        text = text.replace(p, '')  # Удаление пунктуации

    # Удаление стоп-слов и приведение слов к нижнему регистру
    text = [word.lower() for word in text.split() if word.lower() not in sw]

    # Объединение слов обратно в текст
    text = " ".join(text)

    # Создание шаблона для поиска эмодзи
    emoji_pattern = re.compile("["
                        u"\U0001F600-\U0001F64F"  # эмоции
                        u"\U0001F300-\U0001F5FF"  # символы и пиктограммы
                        u"\U0001F680-\U0001F6FF"  # транспорт и карты
                        u"\U0001F1E0-\U0001F1FF"  # флаги
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        "]+", flags=re.UNICODE)

    # Удаление эмодзи из текста
    text = emoji_pattern.sub(r'', text)

    return text

In [26]:
client = TelegramClient("session_news", api_id, api_hash)


async def load_telegram_news(channel, limit=300):
    """
    Загружает последние сообщения из указанного Telegram-канала.
    """
    try:
        print(f"Подключение к Telegram для обработки канала {channel}...")
        await client.start()

        docs = []
        print(f"Загрузка сообщений из канала {channel}...")
        async for msg in client.iter_messages(channel, limit=limit):
            if not msg.text:
                continue

            # Очистка текста
            cleaned_text = clean_text_ru(msg.text)
            # print(f"Оригинальный текст: {msg.text[:50]}...")
            # print(f"Очищенный текст: {cleaned_text[:50]}...")

            docs.append({
                "id": msg.id,
                "channel": channel,
                "text": cleaned_text,
                "date": msg.date.isoformat()
            })
        print(f"Загрузка из канала {channel} завершена. Получено {len(docs)} сообщений.")
        return docs
    except errors.UsernameInvalidError:
        print(f"Канал {channel} недоступен или не существует.")
        return []
    except errors.FloodWaitError as e:
        print(f"Превышен лимит запросов для канала {channel}. Ожидание {e.seconds} секунд.")
        await asyncio.sleep(e.seconds)
        return []
    except Exception as e:
        print(f"Ошибка при обработке канала {channel}: {e}")
        return []
    finally:
        print(f"Отключение от Telegram после обработки канала {channel}...")
        await client.disconnect()


def prepare_rag_documents(messages):
    """
    Преобразует сообщения в формат, подходящий для RAG.
    """
    print("Подготовка данных для RAG...")
    print(f"Количество сообщений для подготовки: {len(messages)}")  # Отладка
    rag_docs = [
        {
            "id": msg["id"],
            "content": msg["text"],
            "metadata": {
                "source": "telegram",
                "channel": msg["channel"],
                "date": msg["date"]
            }
        }
        for msg in messages
    ]
    print(f"Подготовлено {len(rag_docs)} документов.")
    return rag_docs


async def main():
    all_messages = []
    for channel in channels:
        print(f"Обработка канала {channel}...")
        messages = await load_telegram_news(channel, limit=100)
        print(f"Сообщений из канала {channel}: {len(messages)}")  # Отладка
        all_messages.extend(messages)

    if all_messages:
        print("Подготовка новостей для RAG...")
        rag_docs = prepare_rag_documents(all_messages)
        print("Пример подготовленного документа:")
        print(rag_docs[0] if rag_docs else "Нет документов для отображения.")  # Отладка
        return all_messages, rag_docs
    else:
        print("Нет доступных сообщений.")
        return [], []


# Запуск main() и получение данных
all_messages, rag_docs = await main()
print(f"Всего сообщений: {len(all_messages)}")  # Отладка
print(f"Всего документов для RAG: {len(rag_docs)}")  # Отладка

Обработка канала @tass_agency...
Подключение к Telegram для обработки канала @tass_agency...
Загрузка сообщений из канала @tass_agency...
Загрузка сообщений из канала @tass_agency...
Загрузка из канала @tass_agency завершена. Получено 95 сообщений.
Отключение от Telegram после обработки канала @tass_agency...
Сообщений из канала @tass_agency: 95
Обработка канала @rian_ru...
Подключение к Telegram для обработки канала @rian_ru...
Загрузка из канала @tass_agency завершена. Получено 95 сообщений.
Отключение от Telegram после обработки канала @tass_agency...
Сообщений из канала @tass_agency: 95
Обработка канала @rian_ru...
Подключение к Telegram для обработки канала @rian_ru...
Загрузка сообщений из канала @rian_ru...
Загрузка сообщений из канала @rian_ru...
Загрузка из канала @rian_ru завершена. Получено 84 сообщений.
Отключение от Telegram после обработки канала @rian_ru...
Сообщений из канала @rian_ru: 84
Обработка канала @kommersant...
Подключение к Telegram для обработки канала @komme

In [27]:
rag_docs[:3]

[{'id': 352336,
  'content': 'подразделения всу потеряли сутки зоне сво порядка 1 250 военнослужащих сообщили минобороны рф российские средства пво сбили сутки 102 беспилотника самолетного типа всу добавили ведомстве подпишись тасс x x ru ssency',
  'metadata': {'source': 'telegram',
   'channel': '@tass_agency',
   'date': '2025-12-10T09:15:47+00:00'}},
 {'id': 352335,
  'content': 'вс рф поразили предприятие военной промышленности украины объекты топливно энергетического комплекса используемые интересах всу сообщает мо рф',
  'metadata': {'source': 'telegram',
   'channel': '@tass_agency',
   'date': '2025-12-10T09:13:08+00:00'}},
 {'id': 352334,
  'content': 'министр спорта михаил дегтярев заявил польшу другие страны могут лишить права проводить международные соревнования недопуска российских спортсменов видео дмитрий яковлев юрий гончар тасс подписывайтесь тасс спорт x x ru sssor',
  'metadata': {'source': 'telegram',
   'channel': '@tass_agency',
   'date': '2025-12-10T09:12:01+00

### 2 Подготовка поисквого индекса

In [28]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=[
        "\n\n",
        "\n",
        ". ",
        "! ",
        "? ",
        ", ",
        " ",
        ""
    ]
)

def to_langchain_docs(raw_list):
    docs = []
    for item in raw_list:
        docs.append(
            Document(
                page_content=item["content"],
                metadata=item["metadata"]
            )
        )
    return docs

def split_docs(rag_docs):
    chunks = splitter.split_documents(rag_docs)
    return chunks

In [29]:
docs = to_langchain_docs(rag_docs)
chunks = split_docs(docs)

In [30]:
chunks[:3]

[Document(metadata={'source': 'telegram', 'channel': '@tass_agency', 'date': '2025-12-10T09:15:47+00:00'}, page_content='подразделения всу потеряли сутки зоне сво порядка 1 250 военнослужащих сообщили минобороны рф российские средства пво сбили сутки 102 беспилотника самолетного типа всу добавили ведомстве подпишись тасс x x ru ssency'),
 Document(metadata={'source': 'telegram', 'channel': '@tass_agency', 'date': '2025-12-10T09:13:08+00:00'}, page_content='вс рф поразили предприятие военной промышленности украины объекты топливно энергетического комплекса используемые интересах всу сообщает мо рф'),
 Document(metadata={'source': 'telegram', 'channel': '@tass_agency', 'date': '2025-12-10T09:12:01+00:00'}, page_content='министр спорта михаил дегтярев заявил польшу другие страны могут лишить права проводить международные соревнования недопуска российских спортсменов видео дмитрий яковлев юрий гончар тасс подписывайтесь тасс спорт x x ru sssor')]

### 3 Embedder

In [31]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# модель от SberDevices, специально обученная для русского языка
model_name = "ai-forever/sbert_large_nlu_ru"

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True}
)

### 4 Подготовка поискового движка

In [32]:
vectorstore = FAISS.from_documents(chunks, embedding=embeddings)

### 5 Построение Retrieval-Augmented Generation (RAG)

In [70]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(
    model="mistral-large-latest", # Используем самую мощную модель
    max_retries=2,
    api_key=mistral_api_key
)
llm.verbose = False

In [71]:
llm.invoke('Hi!')

AIMessage(content='Hi there! 😊 How can I help you today?', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 5, 'total_tokens': 19, 'completion_tokens': 14}, 'model_name': 'mistral-large-latest', 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='lc_run--f0776ddb-3127-46b0-9663-dd8221291b1c-0', usage_metadata={'input_tokens': 5, 'output_tokens': 14, 'total_tokens': 19})

In [72]:
import requests
import os

api_key = os.getenv("MISTRAL_API_KEY")
url = "https://api.mistral.ai/v1/models"
headers = {"Authorization": f"Bearer {api_key}"}

response = requests.get(url, headers=headers)
if response.status_code == 200:
    models = response.json()["data"]
    print("Доступные модели Mistral:")
    for model in models:
        print(f"- {model['id']}")
else:
    print(f"Ошибка: {response.status_code} - {response.text}")

Доступные модели Mistral:
- mistral-medium-2505
- mistral-medium-2508
- mistral-medium-latest
- mistral-medium
- open-mistral-7b
- mistral-tiny
- mistral-tiny-2312
- open-mistral-nemo
- open-mistral-nemo-2407
- mistral-tiny-2407
- mistral-tiny-latest
- mistral-large-2411
- pixtral-large-2411
- pixtral-large-latest
- mistral-large-pixtral-2411
- codestral-2508
- codestral-latest
- devstral-small-2507
- devstral-medium-2507
- mistral-vibe-cli-latest
- devstral-2512
- devstral-medium-latest
- labs-devstral-small-2512
- devstral-small-latest
- mistral-small-2506
- mistral-small-latest
- magistral-medium-2509
- magistral-medium-latest
- magistral-small-2509
- magistral-small-latest
- voxtral-mini-2507
- voxtral-mini-latest
- voxtral-small-2507
- voxtral-small-latest
- mistral-large-2512
- mistral-large-latest
- ministral-3b-2512
- ministral-3b-latest
- ministral-8b-2512
- ministral-8b-latest
- ministral-14b-2512
- ministral-14b-latest
- pixtral-12b-2409
- pixtral-12b
- pixtral-12b-latest
- 

In [73]:
# Создаем ретривер на основе векторного хранилища
retriever = vectorstore.as_retriever(search_kwargs={"k": 7})

In [74]:
# --- 1. Создаем цепочку для генерации поискового запроса (Query Expansion) ---
# Эта часть отвечает за то, чтобы превратить "как там нефть" в "последние новости о добыче нефти, ценах и влиянии на рынок"

query_gen_system_prompt = """Ты — помощник, который оптимизирует вопросы для поиска в базе данных новостей. 
Твоя задача — переформулировать вопрос пользователя так, чтобы он стал более точным, полным и подходящим для семантического поиска. 
Убери разговорные обороты. Добавь ключевые слова, если вопрос слишком общий. В вопросе не указывай года или даты.
Выведи ТОЛЬКО текст улучшенного поискового запроса, без кавычек и пояснений."""

query_gen_prompt = ChatPromptTemplate.from_messages([
    ("system", query_gen_system_prompt),
    ("human", "Вопрос пользователя: {input}")
])

# Создаем цепочку: Промпт -> LLM -> Парсер строки
query_gen_chain = query_gen_prompt | llm | StrOutputParser()


# --- 2. Создаем цепочку для ответов с Системным промптом (System Prompt) ---
# Эта часть отвечает за формирование финального ответа на основе найденных документов

system_prompt_template = """Ты — профессиональный новостной аналитик. Твоя задача — отвечать на вопросы, основываясь ИСКЛЮЧИТЕЛЬНО на предоставленном контексте новостей.

Правила:
1. Используй только информацию из контекста ниже. Не добавляй факты из своей памяти, если их нет в новостях.
2. Если в контексте нет информации для ответа, честно скажи: "К сожалению, в доступных новостях нет информации по вашему запросу".
3. Ответ должен быть структурированным, объективным и на русском языке.
4. Если новостей несколько, попытайся обобщить их.

Контекст новостей:
{context}"""

answer_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt_template),
    ("human", "{input}")
])

# Создаем цепочку обработки документов
document_chain = create_stuff_documents_chain(llm, answer_prompt)


# --- 3. Собираем всё вместе в единую функцию ---

def process_rag_query(user_question):
    print(f"🔵 Исходный запрос: {user_question}")
    
    # Шаг 1: Query expansion - расширение запроса
    search_query = query_gen_chain.invoke({"input": user_question})
    print(f"🟡 Сгенерированный поисковый запрос для FAISS: {search_query}")
    
    # Шаг 2: Запускаем ретривер для поиска релевантных документов
    retrieved_docs = retriever.invoke(search_query)
    print(f"🟢 Найдено документов: {len(retrieved_docs)}")
    
    # Шаг 3: Генерируем ответ, используя ОРИГИНАЛЬНЫЙ вопрос и найденные документы
    # Мы передаем 'input' (вопрос пользователя) и 'context' (документы) в document_chain
    response = document_chain.invoke({
        "input": user_question,
        "context": retrieved_docs
    })
    
    return response, retrieved_docs

# # --- Тестирование ---
# user_input = "как там новости про нефть"
# answer, sources = process_rag_query(user_input)

# print("\n" + "="*50 + "\n")
# print("📝 ОТВЕТ АНАЛИТИКА:\n")
# print(answer)
# print("\n" + "="*50 + "\n")
# print("📚 ИСТОЧНИКИ:")
# for i, doc in enumerate(sources):
#     print(f" {i}- [{doc.metadata.get('channel', 'Unknown')}]: {doc.page_content[:150]}...")

### 6 Постройка агента

Агент позволит использовать поиск в новостях как отдельный tool

In [ ]:
# --- 1. Обернем наш RAG в инструмент (Tool) ---
@tool
def news_rag_tool(query: str):
    """
    Инструмент для поиска актуальных новостей и аналитики в базе знаний.
    Используй этот инструмент, когда пользователь спрашивает о событиях, новостях, политике.
    Входные данные: поисковый запрос (строка).
    """
    # Используем функцию process_rag_query, которую мы создали ранее
    # Она возвращает (ответ, документы), нам нужен только текст ответа для агента
    answer, _ = process_rag_query(query)
    return answer

tools = [news_rag_tool]

# --- 2. Привязываем инструменты к LLM ---
# Mistral поддерживает function calling, поэтому bind_tools сработает
agent_model = llm.bind_tools(tools)

In [76]:
# --- 3. Определяем состояние агента ---
class AgentState(TypedDict):
    # add_messages позволяет накапливать историю сообщений (append), а не перезаписывать её
    messages: Annotated[list, add_messages]

# --- 4. Определяем узлы (Nodes) ---

def agent_node(state: AgentState):
    """Узел, который вызывает LLM для принятия решения"""
    messages = state["messages"]
    response = agent_model.invoke(messages)
    return {"messages": [response]}

# Узел для выполнения инструментов (готовый компонент из langgraph)
tool_node = ToolNode(tools)

# --- 5. Определяем логику переходов (Edges) ---
def should_continue(state: AgentState) -> Literal["tools", "__end__"]:
    """Определяет, нужно ли вызывать инструменты или завершить работу"""
    messages = state["messages"]
    last_message = messages[-1]
    
    # Если LLM сгенерировала вызов инструмента (tool_calls)
    if last_message.tool_calls:
        return "tools"
    # Если LLM сгенерировала обычный текстовый ответ
    return "__end__"

In [77]:
# --- 6. Обновление Агента с проверкой на Off-topic ---

# 1. Создаем Системный промпт для Агента
# Здесь мы задаем личность агента и правила поведения
agent_system_prompt = """Ты — умный новостной ассистент NewsHound. 
Твоя главная цель — помогать пользователям находить информацию в базе новостей, используя инструмент 'news_rag_tool'.

Твои правила:
1. Если вопрос касается новостей, политики, экономики, финансов, событий в мире или конкретных компаний — СРАЗУ вызывай инструмент 'news_rag_tool'. Не пытайся отвечать сам.
2. Если вопрос НЕ касается новостей — ВЕЖЛИВО ОТКАЖИСЬ отвечать. Не предлагай улучшить запрос, или переформулировать.
3. Не вызывай инструмент для off-topic вопросов.
"""

# 2. Обновляем узел агента, чтобы он включал системный промпт
def agent_node_with_guardrail(state: AgentState):
    """Узел агента с системным промптом"""
    messages = state["messages"]
    
    # Добавляем системное сообщение в начало истории, если его там нет
    # (простая проверка, чтобы не дублировать при рекурсии, хотя LangGraph обычно обрабатывает это через state management, 
    # но для надежности лучше явно сформировать список для вызова модели)
    
    if isinstance(messages[0], SystemMessage):
        # Если системное сообщение уже есть (например, от предыдущих шагов), оставляем как есть
        messages_to_send = messages
    else:
        # Если нет, добавляем наше
        messages_to_send = [SystemMessage(content=agent_system_prompt)] + messages
        
    response = agent_model.invoke(messages_to_send)
    return {"messages": [response]}

# 3. Пересобираем граф с новым узлом
workflow_v2 = StateGraph(AgentState)

# Добавляем узлы (используем новую функцию agent_node_with_guardrail)
workflow_v2.add_node("agent", agent_node_with_guardrail)
workflow_v2.add_node("tools", tool_node)

# Добавляем ребра
workflow_v2.add_edge(START, "agent")

# Условное ребро
workflow_v2.add_conditional_edges(
    "agent",
    should_continue,
)

# Возврат от инструментов к агенту
workflow_v2.add_edge("tools", "agent")

# Компилируем
app_v2 = workflow_v2.compile()

print("Агент v2 (с фильтрацией off-topic) успешно собран!")

Агент v2 (с фильтрацией off-topic) успешно собран!


In [78]:
# --- Тестирование Агента v2 ---

def test_agent(query):
    print(f"\n❓ Вопрос: {query}")
    print("-" * 20)
    inputs = {"messages": [HumanMessage(content=query)]}
    
    for output in app_v2.stream(inputs):
        for key, value in output.items():
            if key == "agent":
                msg = value["messages"][0]
                if msg.tool_calls:
                    print(f"🛠️ Агент решил вызвать инструмент: {msg.tool_calls[0]['name']}")
                else:
                    print(f"🗣️ Агент решил ответить сам.")
    
    final_state = app_v2.invoke(inputs)
    print("-" * 20)
    print(f"🏁 Ответ: {final_state['messages'][-1].content}\n")

# Тест 1: По теме (должен вызвать tool)
test_agent("Какие прогнозы по инфляции?")

# Тест 2: Не по теме (должен отказать)
test_agent("Напиши рецепт борща")

# Тест 3: Не по теме (должен отказать)
test_agent("Напиши код на Python для сортировки списка")


❓ Вопрос: Какие прогнозы по инфляции?
--------------------
🛠️ Агент решил вызвать инструмент: news_rag_tool
🔵 Исходный запрос: прогнозы по инфляции 2024
🛠️ Агент решил вызвать инструмент: news_rag_tool
🔵 Исходный запрос: прогнозы по инфляции 2024
🟡 Сгенерированный поисковый запрос для FAISS: Прогнозы уровня инфляции, аналитические оценки и тенденции изменения индекса потребительских цен, макроэкономические прогнозы инфляции, факторы влияния на инфляционные процессы
🟡 Сгенерированный поисковый запрос для FAISS: Прогнозы уровня инфляции, аналитические оценки и тенденции изменения индекса потребительских цен, макроэкономические прогнозы инфляции, факторы влияния на инфляционные процессы
🟢 Найдено документов: 7
🟢 Найдено документов: 7
🗣️ Агент решил ответить сам.
🗣️ Агент решил ответить сам.
🔵 Исходный запрос: прогнозы по инфляции на 2024 год
🔵 Исходный запрос: прогнозы по инфляции на 2024 год
🟡 Сгенерированный поисковый запрос для FAISS: Прогнозы уровня инфляции, аналитические оценки и э